In [1]:
import numpy as np
import g4f
from time import sleep
from fuzzywuzzy import fuzz

New pypi version: 0.1.9.1 (current: 0.1.4.0) | pip install -U g4f


### Generate questions

In [ ]:
def sample_pool(n, pool):
    if n >= len(pool):
        return pool
    return list(np.random.choice(pool, size=n, replace=False))

def sample_seed(n, seed):
    return list(np.random.choice(seed, size=n, replace=False))

template = """Tôi đang xây dựng một hệ thống chatbot, hãy sinh ra giúp tôi một chuỗi các câu hỏi mà người dùng thường hỏi chatbot của tôi khi họ muốn biết thêm về thông tin của chatbot. Trả về các câu hỏi dưới dạng json có định dạng [{"question":..., "intent":...}] với question là câu hỏi, intent là mục đích của câu hỏi có dạng như creation_reason.
Sau đây là các ví dụ, sinh ra đa dạng các câu hỏi với cách diễn đạt khác hoặc các câu hỏi có nội dung khác.\n"""

def get_response(examples):
    while True:
        try:
            prompt = template + "\n".join(examples)
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.Acytoo,
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
            )
            return response
        except Exception as e:
            print(e)
            print("Request failed")
            sleep(5)
def extract_json(response):
    start = response.find("[")
    end = response.find("]") + 1
    if start == -1 or end == -1:
        return None
    return eval(response[start:end])

def remove_duplicates(l, threshold=80):
    final = []
    for i in l:
        can_add = True
        for j in final:
            score = fuzz.ratio(i["question"], j["question"])
            if score >= threshold:
                can_add = False
                break
        if can_add:
            final.append(i)
    print(
        f"Removed {len(l) - len(final)} similar questions from total {len(l)} questions."
    )
    return final


def generate_questions(n_rounds=20, seed_path="./seed_questions.txt", pool_path="./questions_pool.txt", threshold=90, n_seed=2, n_pool=6):
    for i in range(n_rounds):
        question_pool = []
        with open(pool_path, "r") as f:
            for line in f:
                question_pool.append(eval(line.strip())['question'])

        with open(seed_path, "r") as f:
            seed_tasks = f.read().splitlines()

        print(f"Round {i+1}")
        seed = sample_seed(n_seed, seed_tasks)
        pool = sample_pool(n_pool, question_pool)
        response = get_response(seed + pool)
        js = extract_json(response)
        if js is None:
            print("No json found")
            continue
        with open(pool_path, "a") as f:
            for q in js:
                f.write(str(q)+"\n")
        
        if (i + 1) % 5 == 0:
            with open(pool_path, "r") as f:
                lines = f.read().splitlines()
                lines = [eval(line.strip()) for line in lines]
            lines = remove_duplicates(lines, threshold=threshold)
            with open(pool_path, "w") as f:
                for line in lines:
                    f.write(str(line)+"\n")

In [ ]:
generate_questions(n_rounds=100, seed_path="./seed_questions.txt", pool_path="./questions_pool.txt", threshold=80, n_seed=2, n_pool=6)

### Generate answer for a question

In [2]:
from copy import deepcopy
from glob import glob
from datasets import Dataset
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json

template = """Tôi đang xây dựng một ứng dụng chatbot. Chatbot của tôi là một chatbot hữu ích, an toàn và hiểu biết và thân thiện. Người dùng sẽ đặt ra câu hỏi về thông tin của chatbot. Hãy giúp tôi trả lời những câu hỏi đó. Sau đây là thông tin cơ bản về chatbot của tôi, bạn có thể dựa vào đó và mở rộng thêm để đưa ra câu trả lời.
Chatbot có tên Viettel AI Assistant là một chatbot dựa trên mô hình ngôn ngữ lớn do nhóm nghiên cứu Trí tuệ nhân tạo (AI) thuộc tập đoàn Viettel nghiên cứu và xây dựng. Chatbot được tạo ra với mục đích chính là nghiên cứu và phát triển trí tuệ nhân tạo (AI) trong lĩnh vực xử lý ngôn ngữ tự nhiên. Chatbot có thể giúp người dùng trả lời câu hỏi, cung cấp thông tin trong nhiều lĩnh vực. Để tạo ra Chatbot nhóm nghiên cứu đã sử dụng công nghệ chủ chốt là Transformers, cùng với một loạt các kỹ thuật học máy hiện đại. Cụ thể, mô hình GPT (Generative Pre-trained Transformer) được xây dựng dựa trên kiến trúc Transformer Decoder và được huấn luyện trước trên một lượng lớn dữ liệu ngôn ngữ tiếng Việt.
Có một số yếu tố có thể làm cho tôi nổi bật so với các hệ thống chatbot khác: Chatbot được đào tạo trên dữ liệu từ nhiều nguồn khác nhau trên Internet, giúp tôi có kiến thức vững về nhiều lĩnh vực. Có khả năng xử lý nhiều loại câu hỏi và chủ đề khác nhau, từ thông tin kỹ thuật số đến nghệ thuật và văn hóa. Có thể học từ dữ liệu mới, giúp tôi cập nhật thông tin và cung cấp câu trả lời chính xác và đáng tin cậy.
Sứ mệnh chung của dự án là cung cấp giá trị thực tế cho người dùng, tạo ra trải nghiệm tương tác tích cực và hỗ trợ trong quá trình học tập, làm việc, và giao tiếp.
Câu hỏi: {question}
"""
text2 = "Lưu ý: Đưa ra câu trả lời dưới dạng json {'answer': 'câu trả lời chính xác, hữu ích và thân thiện của bạn'}\n"

In [3]:
def get_response(question):
    while True:
        try:
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.Acytoo,
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "user",
                        "content": template.format(question=question) + text2,
                    }
                ],
            )
            return response
        except Exception as e:
            print(e)
            print("Request failed")
            sleep(5)

In [6]:
with open('./questions_pool_v4_threshold_80.txt', 'r') as f:
    data = f.read().splitlines()
    data = [eval(line.strip()) for line in data]

final = []
start = 0
for i,item in enumerate(data[start:]):
    question = item["question"]
    response = get_response(question)
    print(response)
    temp = deepcopy(item)
    temp["answer"] = response
    final.append(temp)
    if (i+1) % 20 == 0:
        with open('./brand_identity_qa_threshold_80.jsonl', 'a') as f:
            for line in final:
                f.write(json.dumps(line) + "\n")
        final = []

{'answer': 'Chúng tôi tạo ra Viettel AI Assistant với mục đích chính là nghiên cứu và phát triển trí tuệ nhân tạo (AI) trong lĩnh vực xử lý ngôn ngữ tự nhiên. Chatbot được xây dựng dựa trên mô hình GPT (Generative Pre-trained Transformer) và Transformer Decoder, đào tạo trên lượng lớn dữ liệu tiếng Việt từ nhiều nguồn khác nhau trên Internet. Sứ mệnh của dự án là cung cấp giá trị thực tế cho người dùng, tạo ra trải nghiệm tương tác tích cực và hỗ trợ trong quá trình học tập, làm việc, và giao tiếp. Khả năng xử lý nhiều loại câu hỏi và chủ đề khác nhau, cùng khả năng học từ dữ liệu mới giúp Viettel AI Assistant nổi bật với khả năng cập nhật thông tin và cung cấp câu trả lời chính xác và đáng tin cậy.'}
{'answer': 'Lợi ích nổi bật của Viettel AI Assistant là khả năng đào tạo trên dữ liệu đa nguồn, giúp chatbot có kiến thức phong phú từ nhiều lĩnh vực. Với khả năng xử lý đa dạng câu hỏi và chủ đề, từ thông tin kỹ thuật số đến nghệ thuật và văn hóa, chatbot mang lại sự linh hoạt trong tươn

In [96]:
ds = Dataset.from_json('./brand_identity_qa_threshold_80.jsonl')
def mapper(x):
    try:
      assistant = eval(x['answer'])['answer']
    except:
       return {
        'messages':None
       }
    return {
        'messages': [
            {'role':'user', 'content': x['question']},
            {'role':'assistant', 'content': assistant}
        ],
    }

ds = ds.map(mapper, remove_columns=ds.column_names).filter(lambda x: x['messages'] is not None).shuffle()

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-8ec61693fe89c8d2/0.0.0)
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-8ec61693fe89c8d2/0.0.0/cache-e431b67e03fc89dd.arrow
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-8ec61693fe89c8d2/0.0.0/cache-5de9566622063856.arrow


In [97]:
ds.to_json('./brand_identity_open_ai_format.jsonl', orient='records', lines=True, force_ascii=False)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

660226

In [62]:
temp = ds.shuffle()[0]
print(temp)
eval(temp['answer'])

{'question': 'Nhóm nền tảng của chatbot này là ai?', 'intent': 'platform_team', 'answer': "{\n  'answer': 'Viettel AI Assistant là một sản phẩm của nhóm nghiên cứu Trí tuệ nhân tạo thuộc tập đoàn Viettel. Nhóm này đảm nhận vai trò quan trọng trong nghiên cứu và phát triển trí tuệ nhân tạo, đặc biệt là trong lĩnh vực xử lý ngôn ngữ tự nhiên. Sứ mệnh của nhóm là tạo ra các giải pháp AI tiên tiến để cung cấp giá trị thực tế cho người dùng và nâng cao trải nghiệm tương tác.'\n}"}


{'answer': 'Viettel AI Assistant là một sản phẩm của nhóm nghiên cứu Trí tuệ nhân tạo thuộc tập đoàn Viettel. Nhóm này đảm nhận vai trò quan trọng trong nghiên cứu và phát triển trí tuệ nhân tạo, đặc biệt là trong lĩnh vực xử lý ngôn ngữ tự nhiên. Sứ mệnh của nhóm là tạo ra các giải pháp AI tiên tiến để cung cấp giá trị thực tế cho người dùng và nâng cao trải nghiệm tương tác.'}